# ADA CAPI Notebook for Project Milestone 2

## Table of Contents:

1. [Loading and Preparing the Data](#load)
    1. [Load Tabular Data](#tabu)
    2. [Clean Tabular Data](#clean)
2. [Data Extraction](#extract)
    1. [Extracting metrics from textual articles](#gen)
        1. [Defining Article Metrics](#define_arti_metrics)
        2. [Extracting Article Metrics](#extracting_arti_metrics)
3. [Data Exploration](#explor)
    1. [Exploring Path Lengths](#paths)
    2. [Exploring Categories in the Paths](#cats)
    3. [Exploring Subject Strength in Articles](#sub)
        1. [Exploring Subject Strength in Connected Articles](#graph_cat)
        2. [Exploring Subject Strength in Finished Path Articles](#graph_cat_fi)
        3. [Exploring Subject Strength in Uninished Path Articles](#graph_cat_unfi)
4. [Data Analysis](#analysis)
    1. [Analysing Article Metrics](#artmet)
        1. [Analysing Article Metrics by Category](#artmet_cat)
        2. [Analysing Article Metrics in Finished vs Unfinished paths](#artmetfu_path)
    2. [Analysing the In-Degree of Targets in Finished vs Unfinished Paths](#ltt)
    3. [Analysing Possible Shortest Path Distances in Finished vs Unfinished Paths](#shortest)
    4. [Backclick Analysis](#bck_main)
        1. [Backclicks Distribution](#bck_distr)
        2. [Backclick Frequency in Finished vs. Unfinished Games](#bck-freq-compare)
        3. [Backclick Rate per Category](#bck-rate-cat)
        4. [Backclick Frequency Impact on Finishing the Game Likelihood](#bck-test)
5. [Putting Everything Together](#everything)
    

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import os
from scipy import stats 

# Helper functions from utils folder
from utils.analysis import t_test_article_metrics, visualize_article_connections_per_category, sorted_category_counts, shortest_path_find, simple_t_test
from utils.preprocessing import (merge_articles_categories, create_category_dictionaries,
                                filter_games, get_backclicked_pages)

# Formatting libraries
import urllib
import datetime as datetime

# Plotting libraries
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
default_colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']

# Imports to perform article analysis
import textstat
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt') # Punkt tokenizer
nltk.download('stopwords') # Common stopwords

# Regression libraries
import statsmodels.formula.api as smf

# Stats libraries
from statistics import mean, median

# Load config and extract variables
import config
DATA_PATH = config.PATH_TO_DATA
PATH_GRAPH_FOLDER = config.PATH_GRAPH_FOLDER
ARTICLE_FOLDER = config.ARTICLE_FOLDER
GENERATED_METRICS = config.GENERATED_METRICS

<a id="load"></a>
## 1 - Loading and Preparing the Data

Note that you can load the data from [here](#checkpoint1).

<a id="tabu"></a>
#### 1.1 - Load Tabular Data

In [ ]:
# load in all data (except wikipedia articles)
finished_paths = pd.read_csv(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "paths_finished.tsv"), sep='\t', skiprows=15, 
                             names=["hashedIpAddress", "timestamp", "durationInSec", "path", "rating"])
unfinished_paths = pd.read_csv(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "paths_unfinished.tsv"), sep='\t', skiprows=16, 
                               names=["hashedIpAddress", "timestamp", "durationInSec", "path", "target", "type"])
edges = pd.read_csv(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "links.tsv"), sep='\t', skiprows=15, names=["start", "end"], encoding="utf-8")
articles = pd.read_csv(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "articles.tsv"), sep='\t', skiprows=12, names=["article"], encoding="utf-8")
categories = pd.read_csv(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "categories.tsv"), sep='\t', skiprows=13, 
                         names=["article", "category"], encoding="utf-8")
shortest_paths = np.genfromtxt(os.path.join(DATA_PATH, PATH_GRAPH_FOLDER, "shortest-path-distance-matrix.txt"), delimiter=1, dtype=np.uint8)

<a id="clean"></a>
#### 1.2 - Clean Tabular Data

In [ ]:
# Clean up url encoding in edge list
display(edges.head())
edges["start"] = edges.start.apply(urllib.parse.unquote)
edges["end"] = edges.end.apply(urllib.parse.unquote)
display(edges.head())

In [ ]:
# Format datetime as datetime object
finished_paths["datetime"] = finished_paths.timestamp.apply(datetime.datetime.fromtimestamp)
unfinished_paths["datetime"] = unfinished_paths.timestamp.apply(datetime.datetime.fromtimestamp)
display(unfinished_paths.head())

In [ ]:
# Clean up url encoding for articles
display(articles.head())
articles["article"] = articles.article.apply(urllib.parse.unquote)
display(articles.head())

In [ ]:
# Clean up url encoding for categories
display(categories.head())
categories["article"] = categories.article.apply(urllib.parse.unquote)
display(categories.head())

In [ ]:
# Identify broad categories of articles
display(categories.head())
categories["broad_category"] = categories["category"].apply(lambda x: x.split(".")[1]) # first entry after subject.
display(categories.head())

In [ ]:
# merge articles and categories
articles_categories = pd.merge(articles, categories, how="left", on="article")
display(articles_categories.head())

# 6 articles without category!
print("Merge introduced {} NAs in category columns:".format(articles_categories.category.isna().sum()))
articles_categories[articles_categories.category.isna()]

In [ ]:
# Convert paths to a readable format (lists) and remove url encoding
finished_paths["path"] = finished_paths["path"].apply(lambda x: x.split(";"))
finished_paths["path"] = finished_paths["path"].apply(lambda x: [urllib.parse.unquote(y) for y in x])

unfinished_paths["path"] = unfinished_paths["path"].apply(lambda x: x.split(";"))
unfinished_paths["path"] = unfinished_paths["path"].apply(lambda x: [urllib.parse.unquote(y) for y in x])

In [ ]:
# Add start and target articles of path
finished_paths["start"] = [path[0] for path in finished_paths["path"]]
finished_paths["target"] = [path[-1] for path in finished_paths["path"]]

unfinished_paths["start"] = [path[0] for path in unfinished_paths["path"]]
unfinished_paths["target"] = unfinished_paths["target"].apply(urllib.parse.unquote)

<a id="extract"></a>

## 2 - Data Extraction

<a id="gen"></a>

#### 2.1 - Extracting metrics from textual articles

<a id="define_arti_metrics"></a>
##### 2.1.1 - Defining Article Metrics

The following metrics are extracted by performing textual pre-processing techniques in the wikipedia articles:
* Total word count: To understand the length of the article.
* Non stopword frequency: To identify words that contribute to the content's meaning.
* Stopword frequency: To identify common words that may not contribute to the content's meaning.
* Average word length: To assess the complexity of the language used.
* Average sentence length: Longer or more complex sentences (based on characters) may contribute to frustration.
* Number of paragraphs: To see if the article's structure plays a role in people giving up.
* Keyword frequency: To identify the most common keywords to understand the article's focus.
* Readability: To see if the ease of reading the article has an impact (metric: Flesch Reading Ease Score) Link: https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests

In [ ]:
def proprocess_article(article_text):
    preprocessed_text = article_text
    preprocessed_text = preprocessed_text.lower()
    preprocessed_text = preprocessed_text.replace("\n   ", " ") # As the articles are not continuous sentences
    return preprocessed_text

def calculate_article_metrics(article_text):
    preprocessed_text = proprocess_article(article_text)

    words = word_tokenize(preprocessed_text)
    sentences = sent_tokenize(preprocessed_text)

    # Calculate total word count
    total_word_count = len(words)

    # Calculate stopword frequency
    stop_words = set(stopwords.words("english"))
    stopwords_count = 0
    unique_words = []
    for word in words:
        if word.isalpha() and word.lower() in stop_words:
            stopwords_count +=1
        if word.isalpha() and word.lower() not in stop_words:
            unique_words.append(word.lower())

    # Calculate average word length
    average_word_length = sum(len(word) for word in words) / total_word_count

    # Calculate average sentence length
    average_sentence_length = sum(len(sentence) for sentence in sentences) / len(sentences)

    # Calculate number of paragraphs (assume every new line \n is paragraph)
    paragraphs_count = preprocessed_text.count('\n') + 1 # Count last paragraph

    # Calculate keyword frequency
    word_freq = nltk.FreqDist(unique_words)
    most_common_words = word_freq.most_common(10)  # Parameter to adjust

    # Calculate readability (Flesch Reading Ease Score) - 100: Easy to read, 0: Very confusing
    readability = textstat.flesch_reading_ease(preprocessed_text)

    return {
        "word_count": total_word_count,
        "non_stopword_count": total_word_count - stopwords_count,
        "non_stopword_percentage": (total_word_count - stopwords_count) / total_word_count,
        "stopword_count": stopwords_count,
        "stopword_percentage": stopwords_count / total_word_count,
        "avg_word_length": average_word_length,
        "avg_sent_length": average_sentence_length,
        "paragraph_count": paragraphs_count,
        "common_words": most_common_words,
        "readability_score": readability,
    }

<a id="extracting_arti_metrics"></a>
##### 2.1.2 - Extracting Article Metrics

The commented code below was used to access the `plaintext_articles` folder and read all articles inside, creating a dataframe with all the metric information (see table below). To reduce runtime, we compute the article metrics once and then read the generated CSV file.


Article Metrics DataFrame Description:
| Metric             | Metric Name                  | Description                                                                                                   |
|-------------------------|-------------------------|---------------------------------------------------------------------------------------------------------------|
| `word_count`            | Total Word Count        | Represents the total word count in the article.                                                                |
| `stopword_count`        | Stopword Count          | Measures the number of words that do not contribute significantly to the content's meaning.                 |
| `stopword_percentage`   | Stopword Frequency      | Measures the percentage of words that do not contribute significantly to the content's meaning.                 |
| `non_stopword_count`    | Non-Stopword Count      | Measures the number of words that contribute to the content's meaning.                              |
| `non_stopword_percentage`| Non-Stopword Frequency  | Measures the percentage of words that contribute to the content's meaning.                              |
| `avg_word_length`       | Average Word Length     | Calculates the average word length (in characters) in the article.                                               |
| `avg_sent_length`       | Average Sentence Length | Calculates the average sentence length (in characters) in the article.                                               |
| `paragraph_count`       | Number of Paragraphs    | Calculates the total number of paragraphs in the article.                                                        |
| `common_words`          | Keyword Frequency       | Extracts the 10 most frequent of keywords.                  |
| `readability_score`     | Readability (Flesch Score)| Utilizes the Flesch Reading Ease Score for assessing readability. [Learn more](https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests) |


In [ ]:
"""
folder_path = os.path.join(DATA_PATH, ARTICLE_FOLDER)
if os.path.exists(folder_path) and os.path.isdir(folder_path):

  article_metrics = pd.DataFrame(columns=["article", "word_count", "non_stopword_count", "non_stopword_percentage", "stopword_count", "stopword_percentage", "avg_word_length", "avg_sent_length", "paragraph_count", "common_words", "readability_score"])

  for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    if os.path.isfile(file_path):
      root, extension = os.path.splitext(file_name)
      readable_file_name = urllib.parse.unquote(root)
      
      with open(file_path, "r", encoding="utf-8") as article:
        metrics = calculate_article_metrics(article.read())

        metrics["article"] = readable_file_name
        article_metrics.loc[len(article_metrics)] = metrics
else:
  raise FileNotFoundError("The specified folder path does not exist or is not a directory.")

article_metrics.to_csv(os.path.join(GENERATED_METRICS, "article_metrics.csv"), index=False)
"""

Loading the article data (as mentioned before, this is done to reduce runtime)

In [ ]:
article_metrics = pd.read_csv(os.path.join(GENERATED_METRICS, "article_metrics.csv"))

In [ ]:
display(article_metrics.info())
display(article_metrics.head())

<a id="explor"></a>

## 3 -Data Exploration

<a id="paths"></a>

#### 3.1 - Exploring Path Lengths

Compare the path lengths between the finished and unfinished paths to detect potential outliers or trends that might influence the analysis.

In [ ]:
# calculate path lengths for finished paths and show summary statistics
finished_paths["path_length"] = finished_paths.path.apply(lambda el: len(el))
finished_paths["path_length"].describe()

In [ ]:
# calculate path lengths for unfinished paths and show summary statistics
unfinished_paths["path_length"] = unfinished_paths.path.apply(lambda el: len(el))
unfinished_paths["path_length"].describe()

In [ ]:
# compare distributions of finished and unfinished paths
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
threshold = 40 # for now we remove some outliers to make the plots meaningful

# note, plotting the density to compare the distributions (meaningful 0 length and 1 length paths?)
ax.set_title("Distribution of Finished vs. Unfinished Paths")
sns.histplot(x=finished_paths.path_length[finished_paths.path_length < threshold], ax=ax, discrete=True, alpha=0.4, label="finished paths", stat="density", color=default_colors[0])
sns.histplot(x=unfinished_paths.path_length[unfinished_paths.path_length < threshold], ax=ax, discrete=True, alpha=0.4, label="unfinished paths", stat="density", color=default_colors[1])
plt.legend()

In [ ]:
# Compare distributions of path lengths across finished, restarted paths and unfinished paths that timed out
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 4), sharey=True)

sns.histplot(x=finished_paths.path_length[finished_paths.path_length < threshold], ax=axes[0], discrete=True, color=default_colors[0])
axes[0].set_title("Finished Paths")

unfinished_restart= unfinished_paths[(unfinished_paths.path_length < threshold) & (unfinished_paths.type == "restart")]
sns.histplot(data=unfinished_restart, x="path_length", ax=axes[1], discrete=True, color=default_colors[0])
axes[1].set_title("Unfinished Paths - Restart")

unfinished_timeout = unfinished_paths[(unfinished_paths.path_length < threshold) & (unfinished_paths.type == "timeout")]
sns.histplot(data=unfinished_timeout, x="path_length", ax=axes[2], discrete=True, color=default_colors[0])
axes[2].set_title("Unfinished Paths - Timeout")

In [ ]:
# Important note: unfinished paths before 2011 are missing
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
ax.set_title("Distribution of Finished vs. Unfinished Paths")
filter = lambda df, col: df[col]
sns.histplot(x=filter(finished_paths, "datetime"), ax=ax, alpha=0.4, label="finished", bins=100, color=default_colors[0])
sns.histplot(x=filter(unfinished_paths, "datetime"), ax=ax, alpha=0.4, label="unfinished", bins=50, color=default_colors[1])
plt.legend()

Unfinished paths before 2011 are not present in the dataset. It is crucial to keep this in mind for subsequent analyses on differences between finished and unfinished paths, since in some cases the results could change if we include all the paths or we exclude those before 2011.

In [ ]:
# correct the duration in seconds
# since the timeout is after 1800 seconds of inactivity, we consider here the correct duration
unfinished_timeout["effectiveDuration"] = unfinished_timeout["durationInSec"] - 1800

In [ ]:
# compare the distributions of game duration
# different distribution for finished, restart and timeout
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
ax.set_title("Distribution of Finished vs. Unfinished Paths")
filter = lambda df, col: df[(df[col] >= 0) & (df[col] < 1000)][col] # chose 1000 for a better visualization
sns.histplot(x=filter(finished_paths, "durationInSec"), ax=ax, alpha=0.4, label="finished", stat="density", bins=100, color=default_colors[0])
sns.histplot(x=filter(unfinished_restart[unfinished_restart.path_length > 1], "durationInSec"), ax=ax, alpha=0.4, label="restart", stat="density", bins=100, color=default_colors[1])
sns.histplot(x=filter(unfinished_timeout[unfinished_timeout.path_length > 1], "effectiveDuration"), ax=ax, alpha=0.4, label="timeout", stat="density", bins=100, color=default_colors[2])
plt.legend()

Note that we have some weird values for `durationInSec`. Further esploration might be appropriate

In [ ]:
print("{} paths are labeled as 'timeout' but their duration is below 1800 seconds".format(len(unfinished_timeout["durationInSec"][unfinished_timeout["durationInSec"] < 1800])))

Note also that there are few articles that are very often chosen as targets (or as start articles). 
Also this could influence further analyses.

In [ ]:
# print frequency articles
# this is computed with respect to all (finished and unfinished) paths
target_counts = (unfinished_restart["target"].value_counts() + finished_paths["target"].value_counts() + unfinished_timeout["target"].value_counts()).sort_values(ascending=False)

target_percentages = target_counts / target_counts.sum()

print("The 10 most frequent targets (frequency)")
display(target_percentages.head(10))

print("The 10 least frequent targets (frequency)")
display(target_percentages.sort_values(ascending=True).head(10))

In [ ]:
# print frequency articles for start
# this is computed with respect to all (finished and unfinished) paths
start_counts = (unfinished_restart["start"].value_counts() + finished_paths["start"].value_counts() + unfinished_timeout["start"].value_counts()).sort_values(ascending=False)

start_percentages = start_counts / start_counts.sum()

print("The 10 most frequent targets (frequency)")
display(start_percentages.head(10))

print("The 10 least frequent targets (frequency)")
display(start_percentages.sort_values(ascending=True).head(10))

Question: why few articles occur very frequently?
- For targets: 5 articles constitute the 14.20% of the total targets
- For starting articles: 5 articles constitute the 12.82% of the total starting articles

Also this requires further checks and deeper analyses.

We see there are quite a few outliers and extreme values for path lengths. For the future analysis we need to consider:
- Games with a path length of 1.
- Games with high path length (max. path length=435).

We plan to address these questions in the analysis, e.g., through a sensitivity analysis: We run our analysis first on the regular data, before checking if we get similar results while removing certain outlier games.

<a id="cats"></a>

#### 3.2 - Exploring Categories in the Paths

We want to look at the occurances of categories in the paths, to gain an understanding of whether certain categories lead to games that are on average easier for people.

In [ ]:
# Seeing which categories are most represented in articles
count_articles = categories.groupby("broad_category").size()

print("Below shows how many articles each of the broad categories are represented by")
display(count_articles)

In [ ]:
# Create dictionaries for easy discovery of what categories an article belongs to
article_to_category, article_to_broad_category = create_category_dictionaries(categories)

In [ ]:
# Count how many times each category has occured as a target in the finished and unfinished paths.
# Note that some articles are represented by multiple categories, which are thus counted extra.

# Creating a dictionary of counts of the broad categories in the finished paths
sorted_cats_f = sorted_category_counts(finished_paths, article_to_broad_category)

# Creating a dictionary of counts of the broad categories in the unfinished paths
sorted_cats_u = sorted_category_counts(unfinished_paths, article_to_broad_category)

# Plotting the results.
ax = plt.barh(list(sorted_cats_f.keys()), sorted_cats_f.values(), label="Finished paths", color=default_colors[0])
ax2 = plt.barh(list(sorted_cats_u.keys()), sorted_cats_u.values(), left=list(sorted_cats_f.values()), label="Unfinished paths", color=default_colors[1])
plt.xlabel("Count")
plt.title("Occurences of categories as targets")
plt.gca().invert_yaxis()
plt.legend()
plt.show()

The plot shows clearly that some categories occur as targets relatively more in finished paths while others have the opposite outcome. For example, "Couuntries" occurs as a target in finished paths multiple times as often as it does in unfinished paths, whereas "Everyday_life" occurs as a target in finished paths only slightly more often than it does in unfinished paths. 

The plot, however, also shows the imbalance in the categories. We have a lot of paths ending in "Geography" and "Science", but very few ending in "Mathematics" and "Art". We can create a plot with better interpretability regarding the influence of categories by charting the empirical likelihood of a target belonging to a certain category not being reached. This is the probability of a game being unfinished ($u$) for a given category $i$, and is calculated as:

$ P(u|i) = \frac{c^u_i}{c^u_i + c^f_i} $


where $c^u_i$ is the count of category i occuring as a target in the unfinished paths, whereas $c^f_i$ is the count of category i occuring as a target in the finished paths.

However, a large part of the imbalance in the number of finished and unfinished paths comes from the fact that data on unfinished paths started to be collected only a couple of years after it did for finished paths. Thus, to have a probability value that makes sense, we need to exclude the finished path data from the time before data for unfinished paths also began to be collected. We will only perform this filtering for this particular analysis, however, as it results in a significant loss of data.

In [ ]:
# Creating a dictionary of counts of the broad categories in the finished paths played after data began to be collected for unfinished paths.

finished_paths_post2011 = finished_paths[finished_paths["datetime"] >= unfinished_paths["datetime"].min()]
sorted_cats_f_p2011 = sorted_category_counts(finished_paths_post2011, article_to_broad_category)

In [ ]:
# Calculating the empirical likelihood that a target belonging to a certain category not being reached.
cat_unfinished_prob = {}
for i in sorted_cats_f:
    cat_unfinished_prob[i] = sorted_cats_u[i]/(sorted_cats_u[i]+sorted_cats_f_p2011[i])

# Plotting the results.
ax = plt.barh(list(cat_unfinished_prob.keys()), cat_unfinished_prob.values(), color=default_colors[0])
plt.xlabel("Probability")
plt.title("Empirical likelihood of not reaching certain categories")
plt.gca().invert_yaxis()
plt.show()

This plot reinforces the possibility that certain categories make for easier games, as we can see that people are empirically more likely to not reach targets belonging to certain categories more than others.. Nonetheless, to establish a proper relationship, we may in the future need to control for certain other variables. For example, it could be that articles in the "Countries" category are simply better connected than those in the "Everyday_life" category. For this, we would need to potentially make use of matching and propensity scores to conduct a proper causal analysis.


<a id="sub"></a>

#### 3.3 - Exploring Subject Strength in Articles

Conducting an exploratory analysis on the relationship between categories of articles, our focus encompasses both neighboring articles (i.e., those directly connected by a link) and start/target articles in both finished and unfinished paths.

In [ ]:
# Removing the 6 articles without category (found in section 1.2)!
articles_categories = articles_categories.dropna(subset=["broad_category"])

<a id="graph_cat"></a>

##### 3.3.1 - Exploring Subject Strength in Connected Articles
Visualizing the strength of the categories for connected articles (those which are connected by an edge).

In [ ]:
# Merging the category information to the articles in the edges dataframe
edge_category = merge_articles_categories(edges, ["start", "end"], articles_categories)

# Visualizing their article relations through categories in a graph
visualize_article_connections_per_category(edge_category, articles_categories, "Article Connections Based on Category (Normalized and Scaled Edges)")

<a id="graph_cat_fi"></a>

##### 3.3.2 - Exploring Subject Strength in Finished Path Articles
Visualizing the strength of the categories for both start and target articles in the finished paths using a graph.

In [ ]:
# Merging the category information to the articles in the finished_paths dataframe
finished_paths_categories = merge_articles_categories(finished_paths, ["start", "target"], articles_categories)

# Visualizing their article relations through categories in a graph
visualize_article_connections_per_category(finished_paths_categories, articles_categories, "Start & Target Article Connections in Finished Path Based on Category (Normalized and Scaled Edges)")

<a id="graph_cat_unfi"></a>

##### 3.3.3 - Exploring Subject Strength in Uninished Path Articles
Visualizing the strength of the categories for both start and target articles in the unfinished paths using a graph.

In [ ]:
# Merging the category information to the articles in the unfinished_paths dataframe
unfinished_paths_categories = merge_articles_categories(unfinished_paths, ["start", "target"], articles_categories)

# Visualizing their article relations through categories in a graph
visualize_article_connections_per_category(unfinished_paths_categories, articles_categories, "Start & Target Article Connections in Unfinished Path Based on Category (Normalized and Scaled Edges)")

<a id="analysis"></a>

## 4 - Data Analysis

<a id="artmet"></a>

#### 4.1 - Analysing Article Metrics

Conducting an analysis on the article metrics extracted in section 2.1. 

<a id="artmet_cat"></a>

##### 4.1.1 - Analysing Article Metrics by Category

Viewing how the different metrics (e.g., word_count, stopword_count, etc.) vary across different categories of articles. These differences are evident by visualizing the metrics in both bar plots and violin plots (see below).

In [ ]:
# Merge articles with their corresponding categories
article_metrics_with_categories = article_metrics.merge(categories, how="left", on=["article"])
display(article_metrics_with_categories.head())

In [ ]:
metrics_to_plot = ['word_count', 'stopword_count', 'stopword_percentage', 'non_stopword_count', 'non_stopword_percentage','avg_word_length', 'avg_sent_length', 'paragraph_count','readability_score']
fig, axes = plt.subplots(nrows=len(metrics_to_plot), ncols=2, figsize=(15, 6 * len(metrics_to_plot)))

for idx, metric in enumerate(metrics_to_plot):
  # Bar plot
  ax_bar = axes[idx, 0]
  sns.barplot(x=article_metrics_with_categories["broad_category"], y=article_metrics_with_categories[metric], errorbar=("ci", 95), ax=ax_bar, palette=default_colors)
  ax_bar.set_xlabel("Category")
  ax_bar.set_ylabel(metric)
  ax_bar.set_title("Mean and CI of {} per Category".format(metric))
  ax_bar.set_xticklabels(ax_bar.get_xticklabels(), rotation=90)

  # Violin plot
  ax_violin = axes[idx, 1]
  sns.violinplot(x=article_metrics_with_categories["broad_category"], y=article_metrics_with_categories[metric], ax=ax_violin, palette=default_colors)
  ax_violin.set_xlabel("Category")
  ax_violin.set_ylabel(metric)
  ax_violin.set_title("Distribution of {} per Category".format(metric))
  ax_violin.set_xticklabels(ax_violin.get_xticklabels(), rotation=90)

plt.tight_layout()
plt.show()

Some observations:
* Interesting observations from the plot reveal that science articles generally tend to be shorter in length, evident in lower word counts and paragraph counts.
* Mathematical article appear to be the least consistent in word length, with a notable high variance.
* Articles related to countries, on average, display the lowest proportion of stopwords (percentage-wise), suggesting they might posses a rich vocabulary.

<a id="artmetfu_path"></a>

##### 4.1.2 - Analysing Article Metrics in Finished vs Unfinished paths

Viewing how the different metrics (e.g., word_count, stopword_count, etc.) vary across start/target articles in finished/unfinished paths. Both bar plots and violin plots are used to understand the distribution our these metrics. We observe remarkable similarity accross all plots, but minor differences are apparent in word count and paragraph count.

In [ ]:
# Show the article metrics per finished and unfinished parths (both for start and end articles)
start_finished_article_metrics = finished_paths.merge(article_metrics_with_categories, how="left", left_on="start", right_on="article")
end_finished_article_metrics = finished_paths.merge(article_metrics_with_categories, how="left", left_on="target", right_on="article")
start_unfinished_article_metrics = unfinished_paths.merge(article_metrics_with_categories, how="left", left_on="start", right_on="article")
end_unfinished_article_metrics = unfinished_paths.merge(article_metrics_with_categories, how="left", left_on="target", right_on="article")

In [ ]:
metrics_to_plot = ["word_count", "stopword_count", "stopword_percentage", "non_stopword_count", "non_stopword_percentage","avg_word_length", "avg_sent_length", "paragraph_count", "readability_score"]
dataframes = [start_finished_article_metrics, start_unfinished_article_metrics, end_finished_article_metrics, end_unfinished_article_metrics]
dataframe_labels = ["Start Finished", "Start Unfinished", "Target Finished", "Target Unfinished"]

fig, axes = plt.subplots(nrows=len(metrics_to_plot), ncols=2, figsize=(15, 6 * len(metrics_to_plot)))

for idx, metric in enumerate(metrics_to_plot):
  data = [df[metric] for df in dataframes]
  
  # Bar plot
  ax_bar = axes[idx, 0]
  sns.barplot(data=data, errorbar=("ci", 95), ax=ax_bar, palette=default_colors)
  ax_bar.set_xlabel("Type of article")
  ax_bar.set_ylabel(metric)
  ax_bar.set_title("Mean and CI of {} per Category".format(metric))
  ax_bar.set_xticklabels(dataframe_labels)

  # Violin plot
  ax_violin = axes[idx, 1]
  sns.violinplot(data=data, ax=ax_violin, palette=default_colors)
  ax_bar.set_xlabel("Type of article")
  ax_violin.set_ylabel(metric)
  ax_violin.set_title("Distribution of {} per Category".format(metric))
  ax_violin.set_xticklabels(dataframe_labels)

plt.tight_layout()
plt.show()

In [ ]:
# Comparing article metric differences in starting articles in finished/unfinished paths.
print("Start Articles (comparing finished vs unfinished):")
t_test_article_metrics(metrics_to_plot, start_finished_article_metrics, start_unfinished_article_metrics)

# Comparing article metric differences in target articles in finished/unfinished paths.
print("\nTarget Articles (comparing finished vs unfinished):")
t_test_article_metrics(metrics_to_plot, end_finished_article_metrics, end_unfinished_article_metrics)


# Comparing article metric differences in (start, target) article pairs in finished/unfinished paths.
# This enables us to identify differences between the articles one starts with and those that need to be finished 
# in both finished/unfinished paths, aiming to uncover potential factors that might impact the player.
print("\nFinished Articles (comparing start vs target):")
t_test_article_metrics(metrics_to_plot, start_finished_article_metrics, end_finished_article_metrics)

print("\nUnfinished Articles (comparing start vs target):")
t_test_article_metrics(metrics_to_plot, start_unfinished_article_metrics, end_unfinished_article_metrics)

| Metrics                    | Start Articles (Finished vs Unfinished) | Target Articles (Finished vs Unfinished) | Finished Articles (Start vs Target) | Unfinished Articles (Start vs Target) |
|----------------------------|-----------------------------------------|------------------------------------------|--------------------------------------|----------------------------------------|
| word_count                  | t-statistic: 1.873, p-value: 0.061       | t-statistic: 36.838, p-value: 0.000      | t-statistic: -30.824, p-value: 0.000 | t-statistic: 8.949, p-value: 0.000     |
| stopword_count              | t-statistic: 1.353, p-value: 0.176       | t-statistic: 34.692, p-value: 0.000      | t-statistic: -30.009, p-value: 0.000 | t-statistic: 8.076, p-value: 0.000     |
| stopword_percentage         | t-statistic: -3.366, p-value: 0.001      | t-statistic: 4.045, p-value: 0.000       | t-statistic: -0.362, p-value: 0.717 | t-statistic: 6.045, p-value: 0.000     |
| non_stopword_count          | t-statistic: 2.131, p-value: 0.033       | t-statistic: 37.599, p-value: 0.000      | t-statistic: -30.943, p-value: 0.000 | t-statistic: 9.317, p-value: 0.000     |
| non_stopword_percentage     | t-statistic: 3.366, p-value: 0.001       | t-statistic: -4.045, p-value: 0.000      | t-statistic: 0.362, p-value: 0.717  | t-statistic: -6.045, p-value: 0.000    |
| avg_word_length             | t-statistic: -3.090, p-value: 0.002      | t-statistic: 10.974, p-value: 0.000     | t-statistic: 2.987, p-value: 0.003  | t-statistic: 14.113, p-value: 0.000   |
| avg_sent_length             | t-statistic: 4.863, p-value: 0.000       | t-statistic: -0.260, p-value: 0.795     | t-statistic: -3.443, p-value: 0.001 | t-statistic: -6.964, p-value: 0.000    |
| paragraph_count             | t-statistic: 0.038, p-value: 0.970       | t-statistic: 37.247, p-value: 0.000     | t-statistic: -29.294, p-value: 0.000| t-statistic: 11.952, p-value: 0.000   |
| readability_score           | t-statistic: 4.652, p-value: 0.000       | t-statistic: -21.100, p-value: 0.000   | t-statistic: -8.953, p-value: 0.000 | t-statistic: -27.779, p-value: 0.000 |

1. **Finished vs Unfinished Start Articles:**
   - The stopword_percentage is significantly lower (and non_stopword_percentage higher) in finished articles than unfinished, suggesting a potential emphasis on more meaningful content. 
   - Finished start articles also tend to have higher avg_sent_length and readability_score, indicating a focus on well-structured and reader-friendly content.

2. **Finished vs Unfinished Target Articles:**
   - Finished target articles exhibit significantly higher values across various metrics, including word_count, stopword_percentage (with lower non_stopword_percentage), avg_word_length, and paragraph_count. 
   - Additionally, they have a significantly lower readability_score, suggesting that finished target articles could be more challenging to comprehend.

3. **Finished Articles (Start, Target):**
   - When comparing finished start and target articles, strong differences emerge in various metrics. 
   - Finished start articles tend to be shorter with lower word_count and paragraph counts, but have a slightly higher readability_score.

4. **Unfinished Articles (Start, Target):**
   - Starting articles exhibit significantly higher values accross all metrics, except avg_sent_length, non_stopword_percentage, and readability_score. 
   - This implies that target articles which players failed to complete, are consistently longer, include more stopword_percentage (therefore have a lower non_stopword_percentage) and have higher avg_word_length. 
   - Interestingly, unfinished start/target articles exhibit the most significant difference in readability scores, prompting us to consider whether this larger gap affects the user's ability to finish the game.



<a id="ltt"></a>

#### 4.2 - Analysing the In-Degree of Targets in Finished vs Unfinished Paths

It is possible that certain paths are easier objectively because their targets have a larger "in-degree", i.e. the number of edges in the graph pointing to it. This would be intuitive: if there are more ways to get to the target, it should be easier to do so. This section explores whether this idea is reflected in the distributions of the in-degrees of the targets in finished and unfinished paths.

In [ ]:
# Counting how many links point to targets in finished and unfinished paths, known as the "in-degree".

finished_paths["links_to_target"] = finished_paths["path"].apply(lambda x: len(edges.loc[edges["end"] == x[-1]]))
unfinished_paths["links_to_target"] = unfinished_paths["target"].apply(lambda x: len(edges.loc[edges["end"] == x]))

In [ ]:
# Checking for targets with an in-degree of 0.

zeros_finished = [1 for x in finished_paths['links_to_target'] if x == 0]
zeros_unfinished = [1 for x in unfinished_paths['links_to_target'] if x == 0]

print("There were {} targets with an in-degree of 0 in the finished paths.".format(sum(zeros_finished)))
print("There were {} targets with an in-degree of 0 in the unfinished paths.".format(sum(zeros_unfinished)))

It is unreasonable that the targets have an in-degree of 0. It is likely that these outcomes occur because the information provided in the data was not updated after the wikipedia graph kept evolving, as one target with 0 in-degree was reached. It is nonetheless believed that the information is accurate in the majority of cases, so we simply ignore the targets with 0 in-degree in future analyses. 

In [ ]:
# We visualise the data below.

finished_in_degree = [x for x in finished_paths['links_to_target'] if x != 0]
unfinished_in_degree = [x for x in unfinished_paths['links_to_target'] if x != 0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.hist(finished_in_degree, bins=50)
ax1.set_xlabel("in-degree")
ax1.set_ylabel("Count")
ax1.set_title("Histogram of in-degree of targets in finished paths")

ax2.hist(unfinished_in_degree, bins=50)
ax2.set_xlabel("in-degree")
ax2.set_ylabel("Count")
ax2.set_title("Histogram of in-degree of targets in unfinished paths")

The histograms above show that the data is clearly skewed heavily to the right. We suspect this may be a power law. We check this below.

In [ ]:
# Building the arrays of for the cumulative distributions of in-degrees:
finished_indegree_cumulative=plt.hist(finished_paths.links_to_target, bins=100, log=True, cumulative=-1, histtype='step', color=default_colors[0])
unfinished_indegree_cumulative=plt.hist(unfinished_paths.links_to_target, bins=100, log=True, cumulative=-1, histtype='step', color=default_colors[1])
plt.close()

# Plotting the CCDF plots of the in-degrees for finished and unfinished paths:
plt.loglog(finished_indegree_cumulative[1][1:], finished_indegree_cumulative[0], label="Finished paths")
plt.loglog(unfinished_indegree_cumulative[1][1:], unfinished_indegree_cumulative[0], label="Unfinished paths")
plt.title('CCDF plot of the in-degree of targets')
plt.ylabel('# of targets (in log scale)')
plt.xlabel('In-degree (in log scale)')
plt.legend()
plt.show()

This analysis shows that the in-degree may not necessarily follow a power law, as the CCDF plot does not have a negative linear slope. We thus try to see if the data is log-normal, be plotting the histogram of the logarithms of the in-degrees of the targets.

In [ ]:
# We visualise the logs of the data below.

logged_finished = [np.log(x) for x in finished_paths['links_to_target'] if x != 0]
logged_unfinished = [np.log(x) for x in unfinished_paths['links_to_target'] if x != 0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.hist(logged_finished, bins=8, color=default_colors[0])
ax1.set_xlabel("log(in-degree)")
ax1.set_ylabel("Count")
ax1.set_title("Histogram of in-degree of targets in finished paths")

ax2.hist(logged_unfinished, bins=8, color=default_colors[0])
ax2.set_xlabel("log(in-degree)")
ax2.set_ylabel("Count")
ax2.set_title("Histogram of in-degree of targets in unfinished paths")

The histograms show that the data may indeed be distributed log-normally. This is why we report both the mean and the median values for these distributions.

In [ ]:
# Printing mean in-degree of the targets in the finished and unfinished paths.
print("The targets that were reached had an in-degree of {:.3f} on average.".format(mean(finished_in_degree)))
print("The targets that were not reached had an in-degree of {:.3f} on average.".format(mean(unfinished_in_degree)))

In [ ]:
# Printing median in-degree of the targets in the finished and unfinished paths.
print("The targets that were reached had a median in-degree of {}.".format(median(finished_in_degree)))
print("The targets that were not reached had a median in-degree of {}.".format(median(unfinished_in_degree)))

As a t-test assums a normal distribution around the mean, we cannot do the t-test for the in-degrees directly, which are heavily skewed. Instead, we do it for the logarithms of the in-degrees.

In [ ]:
# Conducting a t-test for the in-degree of targets for finished and unfinished paths.
simple_t_test(logged_finished, logged_unfinished)

The p-value of a t-test between the number of links pointing to the targets of finished and unfinished paths is 0.0. This means we reject the null hypothesis that the number of links pointing to the targets are statistically the same at the 5% level of significance, indicating that the in-degree of the target indeed may have a statistical significance in whether a game will be finished or not.

In [ ]:
# Creating a box plot of the trends.

finished_links =  pd.DataFrame()
finished_links["links_to_target"] = finished_in_degree
finished_links["path_type"] = "Finished paths"

unfinished_links =  pd.DataFrame()
unfinished_links["links_to_target"] = unfinished_in_degree
unfinished_links["path_type"] = "Unfinished paths"

df_links = pd.concat([finished_links,unfinished_links])

ax = sns.boxplot(x="path_type", y="links_to_target", data=df_links, palette=default_colors)
plt.xlabel(" ")
plt.ylim([-5,200])
plt.ylabel("In-degree of target")

The boxplots above highlight these conclusions. The in-degree of targets in the finished paths are noticeably higher than those in the unfinished paths.

<a id="shortest"></a>

#### 4.3 - Analysing Possible Shortest Path Distances in Finished vs Unfinished Paths

Another potential factor that may determine whether a game will be completed or not, in a more objective manner, is the shortest path length possible between the source and the target. This factor is also intuitive. If a shorter path exists in theory, the path length should also be shorter on average in practice, leading to simpler games. This section explores whether this idea is reflected in the distributions of the length of the shortest possible paths in finished and unfinished games.

In [ ]:
# Retrieving the shortest possible paths for the finished games.

finished_paths["shortest_path_length"] = finished_paths["path"].apply(
    lambda x: shortest_paths[articles.loc[articles['article'] == x[0]].index[0]][articles.loc[articles['article'] == x[-1]].index[0]]
    )

Important note: There are typos in the targets.

Eg. At index 141 in unfinished paths, the target is written as "Long_peper", when it should be "Long_pepper".

Overall, an issue arises in unfinished paths 28 times, but this doesn't seem to be an issue in finished paths. These data points are ignored so far.

In [ ]:
# Retrieving the shortest possible paths for the unfinished games.

shortest_unfinished, not_found_unfinished = shortest_path_find(unfinished_paths, articles, shortest_paths)

unfinished_paths["shortest_path_length"] = shortest_unfinished
print(f"{not_found_unfinished} shortest paths not found")

In [ ]:
# Testing to confirm that there are no issues in the finished paths.

shortest_finished, not_found_finished = shortest_path_find(finished_paths, articles, shortest_paths)

print(f"{not_found_finished} shortest paths not found")

In [ ]:
# Checking for "impossible" paths.

print("There is {} impossible finished path.".format(len(finished_paths[finished_paths['shortest_path_length'] == 255])))
print("There are {} impossible unfinished paths.".format(len(unfinished_paths[unfinished_paths['shortest_path_length'] == 255])))

Similar to the case where there were targets with an in-degree of 0, it is likely that "impossible" paths exist because the shortest paths matrix provided in the data was not updated after the wikipedia graph kept evolving, as one impossible path was completed. As we determine the shortest possible path length based on that matrix, its errors reflect on our analysis. It is nonetheless believed that the matrix is accurate in the majority of cases, so we simply ignore the "impossible" paths in future analyses. 

In [ ]:
# We visualise the data below.
shortest_possible_finished = finished_paths[finished_paths['shortest_path_length'] != 255]["shortest_path_length"]
shortest_possible_unfinished = unfinished_paths[unfinished_paths['shortest_path_length'] != 255]["shortest_path_length"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.hist(shortest_possible_finished, bins=[1, 2, 3, 4, 5, 6, 7, 8], color=default_colors[0])
ax1.set_xlabel("Length of shortest possible path")
ax1.set_ylabel("Count")
ax1.set_title("Shortest path length possible from source to target in finished paths")

ax2.hist(shortest_possible_unfinished, bins=[1, 2, 3, 4, 5, 6, 7, 8], color=default_colors[0])
ax2.set_xlabel("Length of shortest possible path")
ax2.set_ylabel("Count")
ax2.set_title("Shortest path length possible from source to target in unfinished paths")

The histogram of the shortest possible path length makes clear that unfinished paths tend to have longer possible shortest lengths. Intuitively, it makes sense that this would be the case.

It also reveals how the shortest possible path length follows a fairly normal distribution, so we may continue with our analysis. The means of the distributions provide sufficient information in this case.

In [ ]:
# Printing mean shortest possible paths in the finished and unfinished paths.

print("The shortest possible paths were {:.3f} long on average in the finished paths.".format(shortest_possible_finished.mean()))
print("The shortest possible paths were {:.3f} long on average in the unfinished paths.".format(shortest_possible_unfinished.mean()))

In [ ]:
# Doing a t test on the shortest path lengths.
simple_t_test(shortest_possible_finished, shortest_possible_unfinished)

The p-value of a t-test between the shortest possible path lengths of finished and unfinished games is 0.0. This means we reject the null hypothesis that the shortest possible game paths are statistically the same across the two groups at the 5% level of significance, and thus the length of the shortest path possible does indeed may have a statistically significant effect on whether a game will be completed or not.

This is an interesting situation. The past two analyses show that the targets are more difficult to get to in the unfinished paths, due to their lower in-degree and the larger value of the possible shortest path to them.

A challenge for us may be to try to isolate whether the difference between whether a path is finished or not can be fully explained by more objective factors like this, or if there is a human component that we can isolate as well, when controlling for factors such as these. Eg, are some categories actually more difficult to get to, or do the differences in the target category distributions in the finished and unfinished paths arise because some categories may be more likely to have longer possible shortest paths to them or have fewer links pointing at them?

<a id="bck_main"></a>

#### 4.4 - Back click analysis

In this chapter we are going to explore the effect of the use of backclicks on the likelihood to complete the search game and reach the target page.<br>
In particular, our hypothesis is that the **use of backclics may be an indicator of likely quitting**. It's easy to think that a backclick can mean the user ended up on a page different from what he/she thought, or maybe the backclicked page is missing the links he/she was thinking to found there. Anyway, let's dive into the analysis. 

NOTE: for the following analyses, we will only consider players who played multiple games, exclude all games shorter than a certain threshold and consider only unfinished games where the player actively quitted, so only the one of type "restart". In order to filter out less meaningful data and allows to group data from multiple games played by the same user.

In [ ]:
# Filter games keeping only data collected from players who played at least 10 games, considering only paths with at least 5 clicks and only restarted games
pers_finished_df, pers_unfinished_df = filter_games(finished_paths, unfinished_paths, min_games=10, min_length=5, type="restart")

<a id="bck_distr"></a>
##### 4.4.1. Back-clicks frequency distribution
How frequently do players back-click in general?

In [ ]:
# Concatenate the two filtered dataframes, adding a column to identify finished and unfinished paths
bck_an_all = pd.concat([
    pers_finished_df.assign(finished=True),
    pers_unfinished_df.assign(finished=False)
], ignore_index=True)[["hashedIpAddress", "path", "finished"]]

# Compute backclicked pages (pages a "<" correponds to) and visited pages (path without the "<")
bck_an_all["backclicked_pages"] = bck_an_all["path"].apply(lambda x: get_backclicked_pages(x))
bck_an_all["visited_pages"] = bck_an_all["path"].apply(lambda x: [el for el in x if el != "<"])


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=bck_an_all["backclicked_pages"].apply(lambda x: len(x) if x is not pd.NA else 0), kde=True, bins=50)
plt.title("Distribution of backclicked pages per game")
plt.xlabel("Number of backclicked pages")
plt.ylabel("Number of games")
plt.show()

As we can see the seethe distribution is highly skewed since most of the games have 0 or nearly 0 backclicked pages, but there are a few games with both a lot of visited and backclicked pages.


To analyze the impact of the backclicks on the likelyhood of quitting the game, we will then consider the **backclick frequency** per game, defined as the number of backclicked pages divided by the number of visited pages in that path.<br>
Let's visualize the back-click frequency distribution in two cases:
- first considering all the games independently
- then grouping and averaging back-click frequency of same player over multiple games

In [ ]:
# Compute backclick frequency for each game as the number of backclicked pages divided by the number of visited pages
bck_an_all["backclick_number"] = bck_an_all["backclicked_pages"].apply(lambda x: len(x))
bck_an_all["visited_number"] = bck_an_all["visited_pages"].apply(lambda x: len(x))

bck_an_all["backclick_freq"] = bck_an_all["backclick_number"] / bck_an_all["visited_number"]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharex=True)

sns.histplot(data=bck_an_all["backclick_freq"], kde=True, bins=30, ax=axs[0])
axs[0].set_title("Distribution of backclick frequency for all games")
axs[0].set_xlabel("back-click-rate")
axs[0].set_ylabel("Number of games")

sns.histplot(data=bck_an_all[["hashedIpAddress", "backclick_freq"]].groupby("hashedIpAddress").mean()["backclick_freq"], kde=True, bins=30, ax=axs[1]) 
axs[1].set_title("Distribution of avg backclick frequency for each player over all their games")
axs[1].set_xlabel("Avg backclick-rate")
axs[1].set_ylabel("Number of players")

plt.show()

This difference between the two distributions suggests that the use of backclicks is a characteristic of the player.

<a id=bck-freq-compare></a>
#### 4.4.2 Back-click frequency distribution in finished vs unfinished paths

Let's now compare the average backclick frequency distribution between finished and unfinished paths

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharey=True, sharex=True)

sns.violinplot(bck_an_all[bck_an_all["finished"] == True].groupby("hashedIpAddress")["backclick_freq"].mean(), ax=axs[0], color=default_colors[0], inner="quart")
sns.violinplot(bck_an_all[bck_an_all["finished"] == False].groupby("hashedIpAddress")["backclick_freq"].mean(), ax=axs[1], color=default_colors[1], inner="quart")

fig.suptitle("Distribution of avg backclick frequency for each player over all their games")
axs[0].set_title("Finished games")
axs[1].set_title("Unfinished games")
axs[0].set_ylabel("Avg backclick-frequency")

plt.show()

This violin plot shows how the average backclick frequency is often higher over unfinished paths, giving us the hint this parameter may be an indicator of a more likely quit of the player. <br>
Still we'll need further analysis (with a statistically significant test) to draw some more precise conclusions.

<a id=bck-rate-cat></a>
#### 4.4.3 Backclick rate per category

Do players backclick more frequently on some categories? Do things change if we consider only finished or unfinished paths?

To start exploring possible answers to these questions, let's define the **back-click rate per category** $c$ for a player $P$ as the number of times that $P$ has baclicked on a page that belongs to the category $c$, divide by the number of times $P$ visited a page of that category. This measure should express how likely is the player $P$ to back-click a page of category $c$.

In [ ]:
# Create a dataframe to count the number of backclicks and visits for each category
bck_an_all["visited_categories"] = bck_an_all["visited_pages"].apply(lambda x: [article_to_broad_category[page][0] for page in x if page in article_to_broad_category.keys()])
bck_an_all["backclicked_categories"] = bck_an_all["backclicked_pages"].apply(lambda x: [article_to_broad_category[page][0] for page in x if page in article_to_broad_category.keys()]
                                                                                        if x is not pd.NA else pd.NA)

backclicks_per_category = pd.merge(
    bck_an_all[["hashedIpAddress", "visited_categories", "finished"]].explode("visited_categories").groupby(["hashedIpAddress", "visited_categories", "finished"]).size().reset_index(name="visits_count"),
    bck_an_all[["hashedIpAddress", "backclicked_categories", "finished"]].explode("backclicked_categories").groupby(["hashedIpAddress", "backclicked_categories", "finished"]).size().reset_index(name="backclicks_count"),
    how="left",
    left_on=["hashedIpAddress", "visited_categories", "finished"],
    right_on=["hashedIpAddress", "backclicked_categories", "finished"],
)

backclicks_per_category.drop(columns=["backclicked_categories"], inplace=True)
backclicks_per_category.rename(columns={"visited_categories": "category"}, inplace=True)
backclicks_per_category["backclicks_count"].fillna(0, inplace=True)
backclicks_per_category["backclick_rate_per_category"] = backclicks_per_category["backclicks_count"] / backclicks_per_category["visits_count"]

In [ ]:
# Create a paired-column bar chart, showing for each category the average backclick-rate for finished and unfinished games
plt.figure(figsize=(14, 8))
sns.barplot(data=backclicks_per_category[["category", "finished", "backclick_rate_per_category"]],
            x="category",
            y="backclick_rate_per_category",
            hue="finished",
            order=backclicks_per_category[backclicks_per_category["finished"] == False][["category", "backclick_rate_per_category"]].groupby("category").mean().sort_values(by="backclick_rate_per_category", ascending=False).index,
            hue_order=[True, False],
            errorbar=("ci", 95),
            errwidth=1.4,
            capsize=0.1,
            estimator=np.mean,)
plt.xticks(rotation=80)
plt.title("Avg backclick-rate per category: finished vs unfinished games")
plt.ylabel("Avg backclick-rate per category", labelpad=20)
plt.xlabel("Category", labelpad=20)
plt.legend(title="Finished game", ncols=2)
plt.show()

The bar chart shows that the average back-click rate is often significantly different for finished and unfinished paths.
Since this data compare finished and unfinished per-category backclicl-rate of all players, we could see that it's likely that a player with a high back-click rate will quit, even if the measure of how likely is this case, depends on the category.

<a id=bck-test></a>
#### 4.4.4 Backclick frequency impact on finishing the game

Let's get back to the starting point: is there a significant relationship between the backclick frequency over a game and the fact this game may have ended or not?

A first answer can be found by looking to the [**point biserial correlation**](https://en.wikipedia.org/wiki/Point-biserial_correlation_coefficient), which is a correlation coefficient, mathematically equivalent to the Pearson correlation, specific for cases where we have one continouous variable and a dichotomous one. <br>
This setting fits our needs, in particular we can consider the following random variables:
- $X=\lbrace1 \text{ if the game is finished, } 0\text{ if the game is not finished}\rbrace$ $\implies$ dichotomous variable
- $Y=\text{backclick\_frequency}$ $\implies$ continuous variable

Let's first consider a simple case where all the different games are considered independently each other, so for this first analysis we won't group games played by the same user.

In [ ]:
stats.pointbiserialr(x=bck_an_all["finished"], y=bck_an_all["backclick_freq"])

We can already see a *small* negative correlation which is statistically significant. <br>
In particular, the fact that the correlation is negative already tells us that the higher the $\text{backclick\_frequency}$ the lower the probability of finishing the game $\Bbb{P}(X=1)$.

Let's now group data from different games of the same player, in particular, we'll take for each player it's average $\text{backclick\_frequency}$ over his/her finished games and over his/her unfinished games. <br>
By doing this we'll be able to tell whether there is a significant difference in the average backclick rate of each player, between his/her finished vs. unfinished games.

In [ ]:
stats.pointbiserialr(
    x=bck_an_all.groupby(["hashedIpAddress", "finished"])["backclick_freq"].mean().reset_index()["finished"],
    y=bck_an_all.groupby(["hashedIpAddress", "finished"])["backclick_freq"].mean().reset_index()["backclick_freq"],
)

This time the correlation is even higher (in absolute value) and still statistically significant, leading us to think that the backclick frequency of a player may be an indicator of likely quitting.

<a id="checkpoint1"></a>

## Checkpoint for dataframe

In [ ]:
# The below code saves the version of the dataframe above:
finished_paths.to_pickle(os.path.join(GENERATED_METRICS, "finished_paths_initial_stats.pkl"))
unfinished_paths.to_pickle(os.path.join(GENERATED_METRICS, "unfinished_paths_initial_stats.pkl"))

In [ ]:
# The below code reads that version of the dataframe from the file:
finished_paths = pd.read_pickle(os.path.join(GENERATED_METRICS, "finished_paths_initial_stats.pkl"))
unfinished_paths = pd.read_pickle(os.path.join(GENERATED_METRICS, "unfinished_paths_initial_stats.pkl"))

<a id="everything"></a>

## 5 - Putting Everything Together
### Building a Logistic Regression  to determine influencing factors on the propensity of a player to give up a game (restart or timeout)
We combine all factors we have explored before to build a regression model to predict whether a player gives up to then interpret the coefficients

In [ ]:
### we first merge the article metrics (categories, word count etc.) to the finished and unfinished paths to create a dataset

# merge and unfinished paths while adding a flag
finished_paths["give_up"] = 0
unfinished_paths["give_up"] = 1
games = pd.concat((finished_paths, unfinished_paths), axis=0)

# drop duplicates in the article column, since one article might belong to more than one main category, some rows are duplicated. For now, we just drop these
article_metrics_with_categories = article_metrics_with_categories.drop_duplicates(subset="article")

# define columns that may be relevant from article metrics:
# some columns are exluded since they are contained in others, or because they are complements (stopword vs non-stopword percentage)
keep = ['article',
        'broad_category',
        'paragraph_count',
        'readability_score',
        "stopword_percentage",
        'avg_word_length',
        'avg_sent_length',
        ]

# merge on start
start_metrics = article_metrics_with_categories[keep].add_prefix("start_")
games = pd.merge(games, start_metrics, how="left", left_on="start", right_on="start_article") # add prefix
print(games.shape) # check results of the merge

# merge on target
target_metrics = article_metrics_with_categories[keep].add_prefix("target_")
games = pd.merge(games, target_metrics, how="left", left_on="target", right_on="target_article") # add prefix
print(games.shape) # check results of the merge

# subset games to only include those with reasonable path lengths - TBD, no filtering for now
# games = games[(games.path_length > 1) & (games.path_length < 30)]


# remove unnecessary columns
to_drop = ['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating',
       'datetime', 'start', 'target', 'path_length', "target_article", "start_article", "type",]
games = games.drop(to_drop, axis=1)
print(games.shape) # check results of the subsetting


In [ ]:
percent_missing = games.isnull().sum() * 100 / len(games)
percent_missing

In [ ]:
# drop all NAs - for the actual analysis we need to investigate more why these are coming from.
# for this proof of concept we just remove them
games = games.dropna(axis=0, how="any")
games

In [ ]:
# create formula for logistic regression
target = "give_up"
predictors = [col for col in games.columns if col != target]
formula = target + " ~ " + " + ".join(predictors)
formula

In [ ]:
# logistic regression model with the full formula (i.e. all relevant predictors)
mod = smf.logit(formula=formula, data=games)
res = mod.fit(maxiter=30)
print(res.summary())

In [ ]:
# regression model with limited predictors: only those visible to a player at the start of the game
formula = 'give_up ~ links_to_target + shortest_path_length + start_broad_category + start_paragraph_count + start_readability_score + start_stopword_percentage + start_avg_sent_length + C(target_broad_category)'
mod = smf.logit(formula=formula, data=games)
res = mod.fit(maxiter=30)
print(res.summary())


The two regression models are largely congruent and offer some interesting initial findings (non-exhaustive):
- some categories have a large and statistically significant influence on the proabability of a player giving up. A few examples:
    - paths starting from *language and literature* or more niche topics like *Design and Technology* increase the propensity to give up
    - a target article in the categories *Geography* or *Countries* strongly decreases the probability. This is consistent with the hypothesis that these are rather "easy" categories, as many links point to them.
- similarly, many of the article related metrics are statistically significant; for instance:
    - the shortest_path_length has a large positive coefficient, indicating that objectively longer paths do lead to more failures
    - more detailed article metrics are statistically relevant, but the effect sizes are quite small (e.g., in-degree of target, readability score etc.)